# Applied Data Science Capstone
## Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week1)
Yader Rafael Carrillo Jaime

Future Physicist and Junior Data Scientist 

## 1) Introduction/Business Problem

Around the world, hundreds of people are trying everyday to open small and medium businesses. No matter in what city they are planing to do it, they will look for the best place with the aim of increase their earnings. The present project, is directed to help future entrepreneurs in order to chose the best location to build their businesses in New York City, through providing data about neighborhoods' characteristics and common venues to set up the venture.

It should be noted that to reach this goal, we need to follow a particular structure to show the results. In this case, we were claimed to follow the typical Data science methodology. I hope to do the best of myself along the project.